## POS tagging using modified Viterbi

### Data Preparation

In [5]:
#Importing libraries
import nltk
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint, time

In [3]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

#checking some tagged data.

In [4]:
print(nltk_data[1:5])

[[('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], [('Rudolph', 'NOUN'), ('Agnew', 'NOUN'), (',', '.'), ('55', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), ('and', 'CONJ'), ('former', 'ADJ'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Consolidated', 'NOUN'), ('Gold', 'NOUN'), ('Fields', 'NOUN'), ('PLC', 'NOUN'), (',', '.'), ('was', 'VERB'), ('named', 'VERB'), ('*-1', 'X'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('of', 'ADP'), ('this', 'DET'), ('British', 'ADJ'), ('industrial', 'ADJ'), ('conglomerate', 'NOUN'), ('.', '.')], [('A', 'DET'), ('form', 'NOUN'), ('of', 'ADP'), ('asbestos', 'NOUN'), ('once', 'ADV'), ('used', 'VERB'), ('*', 'X'), ('*', 'X'), ('to', 'PRT'), ('make', 'VERB'), ('Kent', 'NOUN'), ('cigarette', 'NOUN'), ('filters', 'NOUN'), ('has', 'VERB'), ('caused', 'VERB'), ('a', 'DET

#As per instruction Splitting the dataset in sample size of 95:5 for training: validation sets.

In [6]:
train_set,test_set = train_test_split(nltk_data,train_size=0.95,test_size=0.05,random_state = 101)

In [12]:
# create list of train and test tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
test_tagged_words = [tup for sent in test_set for tup in sent]
print(len(train_tagged_words))
print(len(test_tagged_words))

95547
5129


In [13]:
# check some of the train tagged words.
train_tagged_words[1:5]

[('confirmed', 'VERB'), ('the', 'DET'), ('filing', 'NOUN'), ('but', 'CONJ')]

In [14]:
# check some of the test tagged words.
test_tagged_words[1:5]

[('company', 'NOUN'), ('said', 'VERB'), ('0', 'X'), ('it', 'PRON')]

In [15]:
# checking unique tags present in training data
tags = {tag for word,tag in train_tagged_words}
print(len(tags))
print(tags)

12
{'DET', 'ADJ', 'NUM', 'CONJ', 'VERB', '.', 'PRON', 'NOUN', 'X', 'ADV', 'PRT', 'ADP'}


In [16]:
# let's check how many words are present in vocabulary
vocab = {word for word,tag in train_tagged_words}
print(len(vocab))

12100


### Build the vanilla Viterbi based POS tagger

#### Function to compute emission probablity

In [18]:
# Emission probablity function
def word_given_tag(word,tag,train_bag= train_tagged_words):
    taglist = [pair for pair in train_bag if pair[1] == tag]
    tag_count = len(taglist)    
    w_in_tag = [pair[0] for pair in taglist if pair[0]==word]    
    word_count_given_tag = len(w_in_tag)    
    
    return (word_count_given_tag,tag_count)

#### Function to compute transition probabilties

In [20]:
# compute transition probabilities of a previous and next tag
def t2_given_t1(t2,t1,train_bag=train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    
    t1_tags = [tag for tag in tags if tag==t1]
    
    count_of_t1 = len(t1_tags)
    
    t2_given_t1 = [tags[index+1] for index in range(len(tags)-1) if tags[index] == t1 and tags[index+1] == t2]
    
    count_t2_given_t1 = len(t2_given_t1)
    
    return(count_t2_given_t1,count_of_t1)

In [21]:
t2_given_t1('NOUN','DET')

(5284, 8281)

In [22]:
t2_given_t1('VERB','ADJ')

(71, 6069)

In [23]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [24]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))

In [25]:
tags_df

,DET,ADJ,NUM,CONJ,VERB,.,PRON,NOUN,X,ADV,PRT,ADP
DET,0.005676,0.204323,0.022220,0.000483,0.039850,0.017993,0.003744,0.638087,0.045405,0.012438,0.000242,0.009540
ADJ,0.004943,0.066403,0.021256,0.016971,0.011699,0.063931,0.000330,0.699621,0.021091,0.004778,0.010710,0.078267
NUM,0.003276,0.034247,0.184932,0.013699,0.018761,0.117332,0.001489,0.350208,0.210542,0.002978,0.026504,0.036033
CONJ,0.121339,0.118085,0.039981,0.000465,0.156671,0.034868,0.058113,0.349140,0.008833,0.055323,0.004649,0.052534
VERB,0.134392,0.064988,0.022851,0.005577,0.169249,0.034934,0.035786,0.110070,0.217506,0.081952,0.030674,0.092022
.,0.173335,0.043963,0.081003,0.057538,0.089095,0.093320,0.066349,0.222242,0.026971,0.052324,0.002427,0.091342
PRON,0.009954,0.073124,0.006508,0.005360,0.485452,0.040965,0.007657,0.210949,0.089969,0.034074,0.013017,0.022971
NOUN,0.012942,0.012248,0.009542,0.042666,0.147667,0.240604,0.004607,0.263564,0.029175,0.017074,0.043397,0.176514
X,0.054742,0.017187,0.002864,0.010662,0.203851,0.163590,0.055538,0.062381,0.076384,0.024984,0.185232,0.142584
ADV,0.069891,0.129182,0.030474,0.006956,0.343491,0.137131,0.014906,0.031467,0.023186,0.080490,0.014243,0.118582


Viterbi Algorithm steps

1. Given a sequence of words.
2. iterate through the sequence.
3. for each word (starting from first word in sequence) calculate the product of emission probabilties and transition probabilties for all possible tags.
4. assign the tag which has maximum probability obtained in step 3 above.
5. move to the next word in sequence to repeat steps 3 and 4 above.

In [26]:
# Vanilla Viterbi Algorithm
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))


In [28]:
# Let's test our Viterbi algorithm on a few sample sentences of test dataset

random.seed(1234)

# choose random 5 sents
rndom = [random.randint(1,len(test_set)) for x in range(5)]

# list of sents
test_run = [test_set[i] for i in rndom]

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]

In [29]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 

accuracy = len(check)/len(tagged_seq)
print('Vanilla Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  21.82012701034546
Vanilla Viterbi Algorithm Accuracy:  89.38053097345133


In [30]:
# let's check the incorrectly tagged words
[j for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0] != j[1]]

[(('Contra', 'DET'), ('Contra', 'NOUN')),
 (('command', 'VERB'), ('command', 'NOUN')),
 (('Honduras', 'DET'), ('Honduras', 'NOUN')),
 (('Sandinista', 'DET'), ('Sandinista', 'NOUN')),
 (('offensive', 'DET'), ('offensive', 'NOUN')),
 (('rebel', 'DET'), ('rebel', 'NOUN')),
 (('Bucking', 'DET'), ('Bucking', 'VERB')),
 (('drew', 'DET'), ('drew', 'VERB')),
 (('Eveready', 'DET'), ('Eveready', 'NOUN')),
 (('*T*-252', 'DET'), ('*T*-252', 'X')),
 (('complaining', 'DET'), ('complaining', 'VERB')),
 (('up', 'ADV'), ('up', 'PRT'))]

### Solve the problem of unknown words

all of unknown words have been tagged as 'NUM', as 'NUM' is the first tag in tag list and is assigned if unknown word is encountered (emission probability =0).

### Viterbi Modification-Technique I

In [31]:
# use transition probability of tags when emission probability is zero (in case of unknown words)

def Viterbi_1(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        p_transition =[] # list for storing transition probabilities
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            p_transition.append(transition_p)
            
        pmax = max(p)
        state_max = T[p.index(pmax)] 
        
      
        # if probability is zero (unknown word) then use transition probability
        if(pmax==0):
            pmax = max(p_transition)
            state_max = T[p_transition.index(pmax)]
                           
        else:
            state_max = T[p.index(pmax)] 
        
        state.append(state_max)
    return list(zip(words, state))

In [32]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi_1(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi_1 Accuracy: ',accuracy*100)

Time taken in seconds:  17.674171924591064
Modified Viterbi_1 Accuracy:  94.69026548672566


#### Adding Tag occurance probability weights: 
Applying weights based on the probability of tag occurance to the transition probabilities of tags and then use the resulting probability for predicting unknown words

In [34]:
# Creating list containing tuples of POS tags and POS tag occurance probability, based on training data
tag_prob = []
total_tag = len([tag for word,tag in train_tagged_words])
for t in tags:
    each_tag = [tag for word,tag in train_tagged_words if tag==t]
    tag_prob.append((t,len(each_tag)/total_tag))

tag_prob

[('DET', 0.08666938784053921),
 ('ADJ', 0.06351847781719991),
 ('NUM', 0.035145007169246546),
 ('CONJ', 0.02251248076862696),
 ('VERB', 0.1351167488251855),
 ('.', 0.11641391147812072),
 ('PRON', 0.0273373313657153),
 ('NOUN', 0.2862674913916711),
 ('X', 0.06576867928872701),
 ('ADV', 0.031597015081582885),
 ('PRT', 0.03176447193527793),
 ('ADP', 0.0978889970381069)]

In [37]:
def Viterbi_1(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        p_transition =[] # list for storing transition probabilities
       
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
            # find POS tag occurance probability
            tag_p = [pair[1] for pair in tag_prob if pair[0]==tag ]
            
            # calculate the transition prob weighted by tag occurance probability.
            transition_p = tag_p[0]*transition_p             
            p_transition.append(transition_p)
            
        pmax = max(p)
        state_max = T[p.index(pmax)] 
        
      
        # if probability is zero (unknown word) then use weighted transition probability
        if(pmax==0):
            pmax = max(p_transition)
            state_max = T[p_transition.index(pmax)]                 
                           
        else:
            state_max = T[p.index(pmax)] 
        
        state.append(state_max)
    return list(zip(words, state))

In [39]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi_1(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi_1 Accuracy: ',accuracy*100)

Time taken in seconds:  21.056157112121582
Modified Viterbi_1 Accuracy:  95.57522123893806


#### We can see accuracy is improved

In [40]:
# let's check the incorrectly tagged words
[j for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0] != j[1]]

[(('command', 'VERB'), ('command', 'NOUN')),
 (('Sandinista', 'VERB'), ('Sandinista', 'NOUN')),
 (('Eveready', 'VERB'), ('Eveready', 'NOUN')),
 (('*T*-252', 'VERB'), ('*T*-252', 'X')),
 (('up', 'ADV'), ('up', 'PRT'))]

##### we can see words have been correctly POS tagged by Viterbi_1 as compared to vanilla Viterbi Algorithm

### Viterbi Modification-Technique II

we further observe that POS tag 'X' can be easily encapsulated in regex rule, so we extract it only based on ruled based tagged

In [41]:
# specify patterns for tagging
patterns = [
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),               # past tense 
    (r'.*es$', 'VERB'),               # verb    
    (r'.*\'s$', 'NOUN'),              # possessive nouns
    (r'.*s$', 'NOUN'),                # plural nouns
    (r'\*T?\*?-[0-9]+$', 'X'),        # X
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'.*', 'NOUN')                   # nouns
]

# rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

In [42]:
# Modification in Viterbi Algorithm : Backoff to rule based tagger in case unknown word is encountered.
def Viterbi_2(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        state_max = rule_based_tagger.tag([word])[0][1]       
       
        
        if(pmax==0):
            state_max = rule_based_tagger.tag([word])[0][1] # assign based on rule based tagger
        else:
            if state_max != 'X':
                # getting state for which probability is maximum
                state_max = T[p.index(pmax)]                
            
        
        state.append(state_max)
    return list(zip(words, state))

In [43]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi_2(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

Time taken in seconds:  21.067126035690308


In [44]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi_2 Accuracy: ',accuracy*100)

Modified Viterbi_2 Accuracy:  97.34513274336283


In [45]:
# let's check the incorrectly tagged words
[j for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0] != j[1]]

[(('command', 'VERB'), ('command', 'NOUN')),
 (('drew', 'NOUN'), ('drew', 'VERB')),
 (('up', 'ADV'), ('up', 'PRT'))]

The following list of words have been correctly POS tagged by Viterbi_2 as compared to vanilla Viterbi Algorithm:

Contra:correctly tagged as NOUN
Honduras:correctly tagged as NOUN
complaining: correctly tagged as VERB
Bucking: correctly tagged as VERB
the following list of words has been correctly tagged by Viterbi_2 as compared to Viterbi_1

Sandinista: correctly tagged as NOUN
Eveready: correctly tagged as NOUN
*T*-252: correctly tagged as 'X'
#### further modification in Viterb_2: 
We know that the rule based tagger assigns 'NOUN' by default if word does not fall in any rule, to correct this let's assign the tags for any such word based purely on transition probability of tags.

So, first we will modify the rule based tagger to output 'NN' instead of 'NOUN' in case word does not satisfy any rules. We also observe that any capitalized word can still be defaulted as 'NOUN' so will add one more rule for that case

In [46]:
# specify patterns for tagging
patterns = [
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),               # past tense 
    (r'.*es$', 'VERB'),               # verb    
    (r'.*\'s$', 'NOUN'),              # possessive nouns
    (r'.*s$', 'NOUN'),                # plural nouns
    (r'\*T?\*?-[0-9]+$', 'X'),        # X
    (r'^-?[0-9]+(.[0-9]+)?$', 'NUM'), # cardinal numbers
    (r'^[A-Z][a-z].*', 'NOUN'),       # NOUN
    (r'.*', 'NN')                     # default
]

# rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

In [47]:
# modified Viterbi
def Viterbi_2(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        p_transition =[] # for storing transition probabilities
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
            # find POS tag occurance probability
            tag_p = [pair[1] for pair in tag_prob if pair[0]==tag ]
            
            # calculate the transition prob weighted by tag occurance probability.
            transition_p = tag_p[0]*transition_p
            p_transition.append(transition_p)
            
        pmax = max(p)
        state_max = rule_based_tagger.tag([word])[0][1] 
        
      
        # getting state for which probability is maximum
        if(pmax==0):
            state_max = rule_based_tagger.tag([word])[0][1] # assign based on rule based tagger
            
            # if unknown word does not satisfy any rule, find the tag with maximum transition probability
            if state_max == 'NN':
                pmax = max(p_transition)
                state_max = T[p_transition.index(pmax)]                 
                
        else:
             if state_max != 'X':
                # getting state for which probability is maximum
                state_max = T[p.index(pmax)] 
        
        state.append(state_max)
    return list(zip(words, state))

In [48]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi_2(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

Time taken in seconds:  18.668055772781372


In [49]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi Algorithm Accuracy: ',accuracy*100)

Modified Viterbi Algorithm Accuracy:  98.23008849557522


we can see accurency is improved significantly.

In [50]:
# let's check the incorrectly tagged words
[j for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0] != j[1]]

[(('command', 'VERB'), ('command', 'NOUN')), (('up', 'ADV'), ('up', 'PRT'))]

The following list of words have been correctly POS tagged by Viterbi_2 as compared to vanilla Viterbi Algorithm:

* Contra:correctly tagged as NOUN
* Honduras:correctly tagged as NOUN
* complaining: correctly tagged as VERB
* Bucking: correctly tagged as VERB

the following list of words has been correctly tagged by Viterbi_2 as compared to Viterbi_1

* Sandinista: correctly tagged as NOUN
* Eveready: correctly tagged as NOUN
* *T*-252: correctly tagged as 'X'
* drew: correctly tagged as VERB

#### Evaluate vanilla Viterbi and modified Viterbi on entire test data

In [51]:
test_tagged_words = [tup[0] for sent in test_set for tup in sent]
test_run_base = [tup for sent in test_set for tup in sent]

In [52]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  2538.277092933655
Modified Viterbi Algorithm Accuracy:  90.91440826671867


In [53]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi_2(test_tagged_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
print('Modified Viterbi Algorithm Accuracy: ',accuracy*100)

Time taken in seconds:  931.9766390323639
Modified Viterbi Algorithm Accuracy:  95.43770715539091


#### Evaluating tagging accuracy

In [54]:
f = open('Test_sentences.txt')

In [55]:
text = f.read()

In [56]:
sample_test_sent = text.splitlines()

In [57]:
f.close()

In [61]:
sample_test_sent = sample_test_sent[:-3]

In [62]:
sample_test_sent

['Android is a mobile operating system developed by Google.',
 'Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.',
 "Google and Twitter made a deal in 2015 that gave Google access to Twitter's firehose.",
 'Twitter is an online news and social networking service on which users post and interact with messages known as tweets.',
 'Before entering politics, Donald Trump was a domineering businessman and a television personality.',
 'The 2018 FIFA World Cup is the 21st FIFA World Cup, an international football tournament contested once every four years.',
 'This is the first World Cup to be held in Eastern Europe and the 11th time that it has been held in Europe.',
 'Show me the cheapest round trips from Dallas to Atlanta']

In [63]:
# list of untagged words
sample_test_words = [word for sent in sample_test_sent for word in sent.split()]

In [64]:
# tagging the test sentences
start = time.time()
sample_tagged_seq = Viterbi(sample_test_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

Time taken in seconds:  17.240669012069702


In [65]:
sample_tagged_seq

[('Android', 'DET'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google.', 'DET'),
 ('Android', 'DET'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'DET'),
 ('worldwide', 'DET'),
 ('on', 'ADP'),
 ('smartphones', 'DET'),
 ('since', 'ADP'),
 ('2011', 'DET'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013.', 'DET'),
 ('Google', 'DET'),
 ('and', 'CONJ'),
 ('Twitter', 'DET'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'DET'),
 ('that', 'ADP'),
 ('gave', 'VERB'),
 ('Google', 'DET'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ("Twitter's", 'DET'),
 ('firehose.', 'DET'),
 ('Twitter', 'DET'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'DET'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('service', 'NOUN'),
 ('on', 'ADP'),
 ('which', 'DET'),
 ('users',

We can see that several words have been misclassified by vanilla Viterbi POS tagger, for example:

* Android as NUM
* Google as NUM
* OS as NUM

In [66]:
# tagging the test sentences
start = time.time()
sample_tagged_seq = Viterbi_2(sample_test_words)
end = time.time()
difference = end-start

print("Time taken in seconds: ", difference)

Time taken in seconds:  17.494478702545166


In [67]:
sample_tagged_seq

[('Android', 'NOUN'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google.', 'NOUN'),
 ('Android', 'NOUN'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'NOUN'),
 ('worldwide', 'NOUN'),
 ('on', 'ADP'),
 ('smartphones', 'VERB'),
 ('since', 'ADP'),
 ('2011', 'NUM'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013.', 'NOUN'),
 ('Google', 'NOUN'),
 ('and', 'CONJ'),
 ('Twitter', 'NOUN'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'NUM'),
 ('that', 'ADP'),
 ('gave', 'VERB'),
 ('Google', 'NOUN'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ("Twitter's", 'NOUN'),
 ('firehose.', 'NOUN'),
 ('Twitter', 'NOUN'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'NOUN'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('service', 'NOUN'),
 ('on', 'ADP'),
 ('which', 'DET

All these cases were correctly POS tagged by Viterbi_2:

* Android as NOUN
* Google as NOUN
* OS as NOUN

### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

The accuracy of vanilla Viterbi Algorithm: 90.91%

The accuracy of modified Viterbi Algorithm: 95.44%

### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

The following cases were incorrectly tagged which got corrected by modified Viterbi Algorithm:

* Contra:correctly tagged as NOUN
* Honduras:correctly tagged as NOUN
* complaining: correctly tagged as VERB
* Bucking: correctly tagged as VERB
* Sandinista: correctly tagged as NOUN
* Eveready: correctly tagged as NOUN
* "*T*-252": correctly tagged as 'X'
* drew: correctly tagged as VERB